In [1]:
# Load pretrained model, and generate new transformed dataset by passing images through pretrained model 
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.datasets import cifar100
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# Instantiate model 
conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(32, 32, 3))
for layer in conv_base.layers:
    layer.trainable = False
    
top_model = Sequential()
top_model.add(Flatten(input_shape=conv_base.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(BatchNormalization())
top_model.add(Dense(256, activation='relu'))
top_model.add(BatchNormalization())
top_model.add(Dense(100, activation='softmax'))
for layer in top_model.layers:
    layer.trainable = True 

final_model = Model(inputs=conv_base.input, outputs=top_model(conv_base.output))
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [3]:
num_classes = 100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [4]:
batch_size = 32
epochs = 50

datagen = ImageDataGenerator(rescale=1./255)
generator_train = datagen.flow(x_train, y_train, batch_size=batch_size)
generator_test = datagen.flow(x_test, y_test, batch_size=batch_size)

In [ ]:
history = final_model.fit_generator(generator=generator_train,
                                    steps_per_epoch=x_train.shape[0] // batch_size,
                                    epochs=epochs,
                                    validation_data=generator_test,
                                    validation_steps=x_test.shape[0] // batch_size
                                   )

Epoch 1/50
1562/1562 [==============================] - 19s 12ms/step - loss: 3.1596 - acc: 0.2426 - val_loss: 2.8442 - val_acc: 0.2939
Epoch 2/50
1562/1562 [==============================] - 18s 11ms/step - loss: 2.7044 - acc: 0.3211 - val_loss: 2.7303 - val_acc: 0.3185
Epoch 3/50
1562/1562 [==============================] - 18s 11ms/step - loss: 2.5636 - acc: 0.3475 - val_loss: 2.6557 - val_acc: 0.3349
Epoch 4/50
1562/1562 [==============================] - 18s 11ms/step - loss: 2.4557 - acc: 0.3676 - val_loss: 2.6274 - val_acc: 0.3425
Epoch 5/50
1562/1562 [==============================] - 18s 11ms/step - loss: 2.3643 - acc: 0.3878 - val_loss: 2.6075 - val_acc: 0.3447
Epoch 6/50
1562/1562 [==============================] - 18s 11ms/step - loss: 2.2926 - acc: 0.4015 - val_loss: 2.6201 - val_acc: 0.3484
Epoch 7/50
1562/1562 [==============================] - 18s 11ms/step - loss: 2.2275 - acc: 0.4154 - val_loss: 2.5997 - val_acc: 0.3514
Epoch 8/50
1562/1562 [==========================